# Predict car prices

### Problem Statement

A Chinese automobile company Geely Auto aspires to enter the US market by setting up their manufacturing unit there and producing cars locally to give competition to their US and European counterparts.

They have contracted an automobile consulting company to understand the factors on which the pricing of cars depends. Specifically, they want to understand the factors affecting the pricing of cars in the American market, since those may be very different from the Chinese market. The company wants to know:
1. Which variables are significant in predicting the price of a car
2. How well those variables describe the price of a car

Based on various market surveys, the consulting firm has gathered a large data set of different types of cars across the America market.

### Business Goal
We are required to model the price of cars with the available independent variables. It will be used by the management to understand how exactly the prices vary with the independent variables. 

They can accordingly manipulate the design of the cars, the business strategy etc. to meet certain price levels. <br /> 
Further, the model will be a good way for management to understand the pricing dynamics of a new market.

In [541]:
# mlflow ui --host 127.0.0.1 --port 5000
# run above in terminal

In [542]:
# Import necessary libraries
import kagglehub
import shutil
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split  # For splitting data
from sklearn.linear_model import LinearRegression  # Linear regression model
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import mlflow
import mlflow.sklearn
from mlflow.models.signature import infer_signature
from scipy.stats import skew

In [543]:
# # Download latest version
# path = kagglehub.dataset_download("hellbuoy/car-price-prediction")
# # print("Path to dataset files:", path)

# # Define a new visible folder
# destination_path = "/Users/user/Desktop/Portfolio/ML/Kaggle_datasets"
# os.makedirs(destination_path, exist_ok=True)

# # Move all files from the hidden folder to the new location
# for file_name in os.listdir(path):
#     print(file_name)
#     shutil.move(os.path.join(path, file_name), destination_path)


# print("Dataset moved to:", destination_path)

In [544]:
# data_dict = {
#     "variable": [
#         "Car_ID", "Symboling", "carCompany", "fueltype", "aspiration", "doornumber",
#         "carbody", "drivewheel", "enginelocation", "wheelbase", "carlength",
#         "carwidth", "carheight", "curbweight", "enginetype", "cylindernumber",
#         "enginesize", "fuelsystem", "boreratio", "stroke", "compressionratio",
#         "horsepower", "peakrpm", "citympg", "highwaympg", "price"
#     ],
#     "description": [
#         "Unique id of each observation (Integer)",
#         "Its assigned insurance risk rating. A value of +3 indicates that the auto is risky, -3 that it is probably pretty safe. (Categorical)",
#         "Name of car company (Categorical)",
#         "Car fuel type i.e. gas or diesel (Categorical)",
#         "Aspiration used in a car (Categorical)",
#         "Number of doors in a car (Categorical)",
#         "Body of car (Categorical)",
#         "Type of drive wheel (Categorical)",
#         "Location of car engine (Categorical)",
#         "Wheelbase of car (Numeric)",
#         "Length of car (Numeric)",
#         "Width of car (Numeric)",
#         "Height of car (Numeric)",
#         "The weight of a car without occupants or baggage. (Numeric)",
#         "Type of engine (Categorical)",
#         "Cylinder placed in the car (Categorical)",
#         "Size of car (Numeric)",
#         "Fuel system of car (Categorical)",
#         "Boreratio of car (Numeric)",
#         "Stroke or volume inside the engine (Numeric)",
#         "Compression ratio of car (Numeric)",
#         "Horsepower (Numeric)",
#         "Car peak rpm (Numeric)",
#         "Mileage in city (Numeric)",
#         "Mileage on highway (Numeric)",
#         "Price of car (Numeric)"
#     ]
# }

# # Converting the dictionary to a pandas DataFrame
# df_data_dict = pd.DataFrame(data_dict)
# df_data_dict.to_csv('/Users/user/Desktop/Portfolio/ML/Kaggle_datasets/CarPrice/data_dict.csv', index=False)

In [545]:
# # A DataFrame for the data dictionary based on "/Users/user/Desktop/Portfolio/ML/Kaggle_datasets/CarPrice"
# # Also https://archive.ics.uci.edu/dataset/10/automobile

# data_dict = pd.read_csv('/Users/user/Desktop/Portfolio/ML/Kaggle_datasets/CarPrice/data_dict.csv')
# data_dict

Add 2 more columns to the car to describe 
1. The influence of each description
2. full meanings of abbreviations if important

In [546]:
# additional_data_dict = { "influence": [
#         "No direct influence; identifier column.",
#         "Lower symboling values are safer; -3 is safest.",
#         "Car company can influence brand perception and price.",
#         "Diesel is often more fuel-efficient; gas has better acceleration.",
#         "Std (naturally aspirated) is simpler, cheaper, and more reliable. Turbo gives more power but may require more maintenance.",
#         "4 doors are typically more practical than 2 doors.",
#         "Carbody affects style, space, and aerodynamics.",
#         "Drivewheel affects traction and handling (e.g., 4WD is better for off-road).",
#         "Front-engine layout is common and efficient; rear-engine provides unique balance and handling.",
#         "Longer wheelbase improves stability; shorter wheelbase increases agility.",
#         "Longer cars often have more interior space.",
#         "Wider cars offer better stability and comfort.",
#         "Taller cars may provide more headroom but higher center of gravity.",
#         "Higher curb weight often means better stability but worse fuel efficiency.",
#         "Enginetype affects performance and maintenance complexity.",
#         "More cylinders generally mean more power but less fuel efficiency.",
#         "Larger engines provide more power but consume more fuel.",
#         "Fuelsystem influences performance and fuel efficiency; e.g., mpfi is more advanced than carburetors.",
#         "Higher boreratio can increase RPMs, favoring performance.",
#         "Larger stroke generally increases torque, favoring efficiency.",
#         "Higher compression ratio improves power and efficiency but requires high-octane fuel.",
#         "Higher horsepower means better acceleration and performance.",
#         "Higher peakrpm allows for more power at high speeds.",
#         "Higher citympg indicates better urban fuel efficiency.",
#         "Higher highwaympg indicates better fuel efficiency for long trips.",
#         "Higher price reflects better quality, features, or brand value."
#     ],
#     "full_meanings": [
#         "N/A",
#         "Insurance risk rating: +3 (high risk) to -3 (low risk).",
#         "N/A",
#         "Fuel type: 'gas' (gasoline) or 'diesel'.",
#         "'Aspiration': 'std' (naturally aspirated) or 'turbo' (turbocharged).",
#         "Number of doors: '2' or '4'.",
#         "Car body styles: sedan, hatchback, etc.",
#         "Drive wheel types: 'fwd' (front-wheel drive), 'rwd' (rear-wheel drive), '4wd' (four-wheel drive).",
#         "Engine location: front or rear.",
#         "Wheelbase: Distance between front and rear axles (inches).",
#         "Car length: Bumper-to-bumper length (inches).",
#         "Car width: Side-to-side width (inches).",
#         "Car height: Ground to roof height (inches).",
#         "Curb weight: Car weight without passengers or luggage (pounds).",
#         "Engine types: 'dohc', 'ohc', etc.",
#         "Number of cylinders in the engine.",
#         "Engine displacement size (cc).",
#         "Fuel system: e.g., 'mpfi' (multi-point fuel injection), '2bbl' (2-barrel carburetor).",
#         "Boreratio: Cylinder diameter to stroke length ratio.",
#         "Stroke: Distance a piston moves inside the cylinder (inches).",
#         "Compression ratio: Ratio of cylinder volume at BDC to TDC.",
#         "Horsepower: Engine power output (hp).",
#         "Peak revolutions per minute (RPM) the engine can reach.",
#         "City miles per gallon (mpg).",
#         "Highway miles per gallon (mpg).",
#         "Car price (USD)."
#     ]
# }

# # Converting the dictionary to a pandas DataFrame
# additional_df_data_dict = pd.DataFrame(additional_data_dict)

In [547]:
# dfs = [df_data_dict, additional_df_data_dict]
# all_df = pd.concat(dfs, axis=1)
# all_df
# # Units are defaulted to US metrics as the data is a US data 

In [548]:
# # Update csv
# all_df.to_csv('/Users/user/Desktop/Portfolio/ML/Kaggle_datasets/CarPrice/data_dict.csv', index=False)

In [ ]:
# Set Pandas to display full column content
pd.set_option('display.max_colwidth', None)
all_data_dict_df = pd.read_csv("/Users/user/Desktop/Portfolio/ML/Kaggle_datasets/CarPrice/data_dict.csv")
all_data_dict_df

In [550]:
pd.reset_option('display.max_colwidth')

## Cleaning

In [ ]:
df = pd.read_csv('/Users/user/Desktop/Portfolio/ML/Kaggle_datasets/CarPrice/CarPrice_Assignment.csv')
df.head(2)

In [ ]:
df.sample(3)

In [ ]:
df.info()
# No empty rows/values

In [ ]:
categorical_columns = ['fueltype', 'aspiration', 'doornumber', 'carbody', 'drivewheel', 'enginelocation', 'enginetype', 'cylindernumber', 'fuelsystem']

for col in categorical_columns:
    print(f"Category in {col} is : {df[col].unique()}")

In [555]:
number_map = {'two':2, 'three':3, 'four':4, 'five':5, 'six':6, 'eight':8, 'twelve':12}

In [556]:
df.replace({"cylindernumber": number_map},inplace=True)
df.replace({"doornumber": number_map},inplace=True)

In [ ]:
df.head(2)

In [558]:
df['CarName'] = df['CarName'].apply(lambda col: col.lower())
df.insert(3, 'BrandName', '')

# Extract the first word from Car Name
df['BrandName'] = df['CarName'].apply(lambda x: str(x).split(' ', 1)[0])

In [ ]:
df.sample(2)

In [560]:
cols_to_edit = ['BrandName', 'CarName']

replacements = {
    'vw': 'volkswagen',
    'vokswagen': 'volkswagen',
    'toyouta': 'toyota',
    'porcshce': 'porsche',
    'maxda' : 'mazda'
}

# Apply replacements
df[cols_to_edit] = df[cols_to_edit].replace(replacements)


In [ ]:
df['BrandName'].value_counts()

In [562]:
del df['CarName']
del df['car_ID']

## EDA

In [ ]:
plt.title('Distribution of Car Prices', fontsize=16)
plt.ylabel('Price (USD)', fontsize=12)
sns.boxplot(y=df['price'], color="Aqua", flierprops=dict(markerfacecolor='red', marker='o'))
plt.show()

In [ ]:
Q1 = df['price'].quantile(0.25)  # First quartile
Q3 = df['price'].quantile(0.75)  # Third quartile
IQR = Q3 - Q1                    # Interquartile range

# Define outlier range
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter outliers
outliers = df[(df['price'] < lower_bound) | (df['price'] > upper_bound)]
print("Number of outlier rows is" , len(outliers))
print("Number of rows in the df is" , len(df))
print("Percentage of outliers in the df is", round((len(outliers)/len(df))*100,2), '%')


In [ ]:
categorical_features = ['fuelsystem', 'fueltype', 'aspiration', 'carbody', 'drivewheel', 'enginelocation', 'enginetype']

plt.figure(figsize=(16, 12))

for i, feature in enumerate(categorical_features):
    value_counts = df[feature].value_counts()
    plt.subplot(3, 3, i + 1)  # 2 rows and 3 columns grid layout
    sns.barplot(x=value_counts.index, y=value_counts.values, color="aqua")  
    plt.title(f'Distribution of {feature}')
    plt.xlabel(feature)
    plt.ylabel('Count')
    plt.xticks(rotation=45)  # Rotate x-axis labels for readability

plt.tight_layout()
plt.show()


In [ ]:
# Distribution of Numerical Features
numerical_features = ['doornumber', 'wheelbase', 'carlength', 'carwidth', 'carheight', 'curbweight', 'cylindernumber', 'enginesize', 'boreratio', 
                      'stroke', 'compressionratio', 'horsepower', 'peakrpm', 'citympg', 'highwaympg', 'price']

plt.figure(figsize=(16, 12))
for i, feature in enumerate(numerical_features):
    plt.subplot(4, 4, i + 1)
    sns.histplot(data=df[feature], bins=20, kde=True, color="aqua")
    plt.title(feature)
plt.tight_layout()
plt.show()

In [ ]:
correlation_matrix = df[numerical_features].corr()
plt.figure(figsize=(25,15))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation of Numerical features')
plt.show()

## Pre-processing Data

In [ ]:
correlation_matrix['price'].sort_values(ascending=False)

In [573]:
# Select features with high correlation with price using the heat map and the list above
Strong_positive_correlation_columns = ['carwidth', 'curbweight', 'enginesize', 'horsepower']
Strong_negative_correlation_columns = ['citympg', 'highwaympg']

low_correlation_columns = ['doornumber', 'stroke', 'compressionratio', 'peakrpm']

# Drop features with very low correlation with price, 
# e.g., doornumber (0.032), stroke (0.079), compressionratio (0.068), peakrpm(-0.085)

# Drop these columns
df = df.drop(columns=low_correlation_columns)

In [574]:
# Drop cylindernumber since it's highly correlated with enginesize
df = df.drop(columns=['cylindernumber'])

##### PCA analysis

In [ ]:
# carlength & width into car dimensions
car_dimensions_df = df[['carlength', 'carwidth']]
print(type(car_dimensions_df))
car_dimensions_df.head()

In [ ]:
# Standardize the data
scaler = StandardScaler()
data_standardized = scaler.fit_transform(car_dimensions_df)

# Perform PCA
pca = PCA(n_components=1)  # We want one principal component
df['car_dimension'] = pca.fit_transform(data_standardized)

print("Explained variance ratio:", pca.explained_variance_ratio_) 
# If the explained variance ratio is high (e.g., > 0.9), it's safe to drop the original columns.

In [577]:
df = df.drop(columns=['carlength', 'carwidth'])

In [ ]:
# citympg/highwaympg into fuel efficiency
fuel_efficiency_df = df[['citympg', 'highwaympg']]
print(type(fuel_efficiency_df))
fuel_efficiency_df.head()

In [ ]:
# Loop through the columns and print sorted unique values
for col in fuel_efficiency_df:
    unique_values = pd.Series(df[col].unique())  # Convert to Series
    print(f"Values in {col} are: {unique_values.sort_values().values}")
    # Check if the range is somewhat the same to perform either Normalization or Standardization

In [ ]:
# Standardize the data
scaler = StandardScaler()
data_standardized = scaler.fit_transform(fuel_efficiency_df)

# Perform PCA
pca = PCA(n_components=1)  # We want one principal component
df['fuel_efficiency'] = pca.fit_transform(data_standardized)

print("Explained variance ratio:", pca.explained_variance_ratio_) 
# If the explained variance ratio is high (e.g., > 0.9), it's safe to drop the original columns.

Both Standardization & normalization for fuel_efficiency_df have explained variance ratio of over 90%

In [581]:
df = df.drop(columns=['citympg', 'highwaympg'])

In [ ]:
# Standardize columns with different scales
cols_to_standardize_df = df[['symboling', 'wheelbase', 'carheight', 'curbweight', 'enginesize', 'boreratio', 'horsepower']]

# Standardize the data
scaler = StandardScaler()
df[cols_to_standardize_df.columns] = scaler.fit_transform(cols_to_standardize_df)

df.head(2)

In [ ]:
df.columns

In [ ]:
df = df[['BrandName', 'fueltype', 'aspiration', 'carbody', 'drivewheel', 'enginelocation', 'enginetype', 'fuelsystem', 
         'symboling', 'wheelbase', 'carheight', 'curbweight', 'enginesize', 'boreratio', 'horsepower', 'car_dimension', 
         'fuel_efficiency', 'price']]

df.head(2)

In [585]:
categorical_features += ['BrandName']

In [ ]:
print(categorical_features)

In [587]:
one_hot_df = pd.get_dummies(df, columns=categorical_features, dtype=int, drop_first=True)

In [ ]:
one_hot_df.sample(2)

## Split Dataset

In [589]:
X = one_hot_df.drop('price', axis=1)  # Features
y = np.log1p(one_hot_df['price'])  # Target variable. Apply log transformation to the target variable

In [590]:
# Split into training and testing sets ()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=23)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Check current tracking URI
print("Tracking URI:", mlflow.get_tracking_uri())

In [ ]:
# Start MLflow tracking
mlflow.set_experiment("car_price_prediction")

In [ ]:
# Model training and logging
with mlflow.start_run():  # Start an MLflow run
    # Log parameters
    mlflow.log_artifact("Car_prices.ipynb")
    mlflow.set_tag("version", "v2.3")
    mlflow.set_tag("notes_outliers", "Transformed skewed columns to deal with outliers. Scaled all skewed columns except 'price'. Reversed the transformations so the target variable is understandable. Changed how I appended BrandName to categorical features as it wasn't appending properly initially")
    model_params = {"fit_intercept": True}
    mlflow.log_params(model_params)

    # Train the model
    model = LinearRegression(**model_params) # Initialize the model
    model.fit(X_train_scaled, y_train) # Train the model on training data

    # Make predictions
    y_pred = model.predict(X_test_scaled)

    # Reverse the log transformation to interpret predictions
    y_pred = np.expm1(y_pred)

    # Calculate metrics
    r2 = r2_score(np.expm1(y_test), y_pred)  # Reverse the log transformation for y_test as well
    mae = mean_absolute_error(np.expm1(y_test), y_pred)
    mse = mean_squared_error(np.expm1(y_test), y_pred)

    # Log metrics
    mlflow.log_metric("R2", r2)
    mlflow.log_metric("MAE", mae)
    mlflow.log_metric("MSE", mse)

    # Infer the model signature from training data
    signature = infer_signature(X_train, model.predict(X_train))

    input_example = X.head(1)

    # Log the model
    mlflow.sklearn.log_model(model, "car_prediction_linear_regression_model", input_example=input_example, signature=signature)

    print(f"Logged metrics: R2={r2}, MAE={mae}, MSE={mse}")


In [ ]:
coefficients = pd.DataFrame({
    "Feature": X_train.columns,
    "Coefficient": model.coef_
}).sort_values(by="Coefficient", ascending=False)

print(coefficients)

In [ ]:
y_pred = model.predict(X_test)
y_pred

In [ ]:
# Visualization
plt.scatter(y_test, y_pred, color="aqua")
m, b = np.polyfit(y_test, y_pred, 1)  # Linear fit (degree 1)
best_fit_line = m * np.array(y_test) + b

# Plot the line of best fit
plt.plot(y_test, best_fit_line, color="red", alpha=0.7)

# Set x-axis to start from 0
plt.xlim(left=0)

plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.title("Actual vs Predicted")
plt.show()

In [ ]:
# Calculate residuals
residuals = y_test - y_pred

plt.scatter(y_pred, residuals, alpha=0.6)
plt.axhline(0, color='red', linestyle='--')  # Reference line at zero
plt.title('Predicted Values vs. Residuals')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.show()
